<a href="https://colab.research.google.com/github/HarishSingh1981/Session4Assignment/blob/main/S4AssignmentSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [38]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3,padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16, 48, 3,padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)
        self.BN1 = nn.BatchNorm2d(16)
        self.BN2 = nn.BatchNorm2d(48)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(48,128)
        self.fc2 = nn.Linear(128,10)
        self.dout = nn.Dropout(0.50)

    def forward(self, x):
        x = self.avgpool(self.BN2(F.relu(self.conv2(self.pool1(self.BN1(F.relu(self.conv1(x))))))))
        #x = self.avgpool((F.relu(self.conv3(x))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
       # print(f'shape of x --> {x.shape}')
        #Taking average along width and height [GAP]
        #x = x.view(x.size(0), x.size(1)//4, -1)
        #print(f'shape of x --> {x.shape}')
        #flattening the channels
        x = x.reshape(-1,48)
        x = self.dout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [39]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 48, 14, 14]           6,960
       BatchNorm2d-5           [-1, 48, 14, 14]              96
 AdaptiveAvgPool2d-6             [-1, 48, 1, 1]               0
           Dropout-7                   [-1, 48]               0
            Linear-8                  [-1, 128]           6,272
            Linear-9                   [-1, 10]           1,290
Total params: 14,810
Trainable params: 14,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.36
P

<ipython-input-38-174cb49474ed>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [24]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [40]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-38-174cb49474ed>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.7835848927497864 batch_id=468: 100%|██████████| 469/469 [00:55<00:00,  8.46it/s]



Test set: Average loss: 0.6322, Accuracy: 8357/10000 (84%)



loss=0.34977254271507263 batch_id=468: 100%|██████████| 469/469 [00:54<00:00,  8.57it/s]



Test set: Average loss: 0.3229, Accuracy: 9105/10000 (91%)



loss=0.2613304555416107 batch_id=468: 100%|██████████| 469/469 [00:53<00:00,  8.72it/s]


...


Test set: Average loss: 0.0755, Accuracy: 9776/10000 (98%)



loss=0.15708096325397491 batch_id=468: 100%|██████████| 469/469 [00:53<00:00,  8.79it/s]



Test set: Average loss: 0.0776, Accuracy: 9770/10000 (98%)



loss=0.10173242539167404 batch_id=468: 100%|██████████| 469/469 [00:54<00:00,  8.66it/s]



Test set: Average loss: 0.0724, Accuracy: 9797/10000 (98%)

